# Visualisation // Animation

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from matplotlib import colors, cm
from ipywidgets import interact, IntSlider
import ipywidgets as widgets
import matplotlib.colors as mcolors

def get_xy(x, y, yparity=1):   # this function returns arrays in range (-r_max, ..., r_max))
    newlen = len(x)*2
    new_x = np.zeros(newlen)
    new_y = np.zeros(newlen)
    
    new_x[:newlen//2] = -x[::-1]
    new_x[newlen//2:] = x
    new_y[:newlen//2] = yparity * y[::-1]
    new_y[newlen//2:] = y
    
    return new_x, new_y

# Parameters
matter = 'rad'
mu = 0.8
fn = f'./DPV_{str(matter)}/visdata_{mu:.3f}.hdf5'

# Load data
with h5py.File(fn, 'r') as f:
    r = f['r'][:]
    t = f['t'][:]
    M = f['M'][:]
    R = f['R'][:]
    drho = f['drho'][:]
    Theta = f['Theta'][:]

rmax = np.max(r) * np.sin(np.pi/4)  
zmin, zmax = [1e-1, 1e1]
N_iterations = len(drho)

# Precompute the grid once
rcord = np.linspace(-rmax, rmax, 2000)
X, Y = np.meshgrid(rcord, rcord)
_rad = (X**2 + Y**2)**0.5



# Interactive function
def plot_iteration(iteration=0):
    # Create the figure
    fig, ax = plt.subplots(1, 1, figsize=(5, 4))


    # rho /rho_bkg
    x = r
    y = drho[iteration]
    x,y = get_xy(x, y) 
    
    zmin, zmax = [1e-1, 1e1]
    drho_intp = interp1d(x, y, kind='quadratic')
    rcord = np.linspace(-rmax, rmax, 1000)
    X, Y = np.meshgrid(rcord,rcord)
    _rad = (X**2+Y**2)**0.5
    Z = drho_intp(_rad)    
    
    cs = ax.pcolormesh(X, Y, Z,  cmap=cm.seismic, norm=colors.LogNorm(vmin=zmin, vmax=zmax))
    
    # colobars
    cbaxes = fig.add_axes([0.05, 0.25, 0.05, 0.45]) 
    cb = plt.colorbar(cs, cax = cbaxes)
    cbaxes.yaxis.set_ticks_position('left')
    cbaxes.yaxis.set_label_position('left')
    cbaxes.set_ylabel(r"$\frac{\rho_E}{\rho_{\rm bkg} }$")
    ax.set_yticks([])


    if True:
        ## Trapped regions ( > Hubble Rad and boundary AH-PBH) 
        x = r
        crit = 2*M/R
        y = crit[iteration]
        x,y = get_xy(x, y) 
    
        zmin, zmax = [1e-1, 1e1]
        crit_intp = interp1d(x, y, kind='quadratic')
        rcord = np.linspace(-rmax, rmax, 1000)
        X, Y = np.meshgrid(rcord,rcord)
        _rad = (X**2+Y**2)**0.5
        Z = crit_intp(_rad)
        
        mask = (Z<1) 
        Z[mask] = np.nan
    
        colors_list = ['green']  # transparent, then black
        cmap = mcolors.ListedColormap(colors_list)
        bounds = [0, 1]
        norm = mcolors.BoundaryNorm(bounds, cmap.N)
        cs = ax.pcolormesh(X, Y, Z, cmap=cmap, norm=norm, alpha = 0.5)

    plt.show()

# Create interactive widget
interact(plot_iteration, 
         iteration=IntSlider(min=0, max=N_iterations-1, step=1, value=1, 
                           description='Iteration:', continuous_update=True))

interactive(children=(IntSlider(value=1, description='Iteration:', max=587), Output()), _dom_classes=('widget-…

<function __main__.plot_iteration(iteration=0)>